In [1]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
import pickle
import time
from tensorflow.keras.callbacks import TensorBoard

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 300MB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)


X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]
# input_shape = (32,50,50,1)
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            

            model = Sequential()
            model.add(Conv2D(layer_size,(3,3), input_shape= X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPool2D(pool_size=(2,2)))


            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPool2D(pool_size=(2,2)))
                
            model.add(Flatten())
            for _ in range(dense_layer):    
                model.add(Dense(512))
                model.add(Activation("relu"))
            
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            with tf.device('/cpu:0'):
                model.fit(X,y,batch_size=32, epochs = 10, validation_split=0.3, callbacks=[tensorboard])
model.summary()

model.save('64x3-CNN.model')


1 Physical GPUs, 1 Logical GPUs
Epoch 1/10
546/546 [==============================] - 60s 110ms/step - loss: 0.6442 - accuracy: 0.6166 - val_loss: 0.5887 - val_accuracy: 0.6992
Epoch 2/10
546/546 [==============================] - 58s 107ms/step - loss: 0.5534 - accuracy: 0.7171 - val_loss: 0.5115 - val_accuracy: 0.7477
Epoch 3/10
546/546 [==============================] - 62s 113ms/step - loss: 0.4983 - accuracy: 0.7591 - val_loss: 0.4825 - val_accuracy: 0.7727
Epoch 4/10
546/546 [==============================] - 60s 110ms/step - loss: 0.4652 - accuracy: 0.7812 - val_loss: 0.5177 - val_accuracy: 0.7519
Epoch 5/10
546/546 [==============================] - 57s 105ms/step - loss: 0.4341 - accuracy: 0.7987 - val_loss: 0.4421 - val_accuracy: 0.7989
Epoch 6/10
546/546 [==============================] - 57s 105ms/step - loss: 0.4036 - accuracy: 0.8147 - val_loss: 0.4522 - val_accuracy: 0.7897
Epoch 7/10
546/546 [==============================] - 57s 105ms/step - loss: 0.3730 - accuracy: 0.